<a href="https://colab.research.google.com/github/hholl19/Deep-Leaning-Class-Notes/blob/hholl19_added_things/Applied_Deep_Learning_2nd_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
import pandas as pd
import numpy as np
import os
import pathlib 
from pathlib import Path
#import cv2
#from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten, Conv2D, MaxPooling2D 
from keras.layers.normalization import BatchNormalization 
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg19 import VGG19

from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from matplotlib import pyplot

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path= '/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs'
folders = os.listdir(file_path)
print(folders)

['test.csv', 'train.csv', 'train', 'test']


In [5]:
df_train= pd.read_csv('/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train.csv')
df_train.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [6]:
df_train["image_id"]="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/" + df_train["image_id"] + ".jpg"
df_train.head()

,image_id,breed
0,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,nerodia-erythrogaster
1,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,pantherophis-vulpinus
2,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,thamnophis-sirtalis
3,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,pantherophis-obsoletus
4,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,agkistrodon-contortrix


In [7]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

80142336/80134624 [==============================] - 0s 0us/step


In [8]:
for layer in vgg19.layers:
    layer.trainable = False

In [9]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.3, name='dropout'))
model.add(Dense(35, activation='softmax', name='predictions'))

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
train_datagen = ImageDataGenerator(rescale=1/255,
        validation_split=0.2,
        data_format='channels_last',
        rotation_range=30,
        horizontal_flip=True,
        fill_mode='reflect') 
test_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [12]:
train_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/",
        x_col="image_id",
        y_col="breed",
        subset="training",
        batch_size=32,
        seed=123,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224)) # Change acc to model requirements

Found 4407 validated image filenames belonging to 35 classes.


In [13]:
test_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/",
        x_col="image_id",
        y_col="breed",
        subset="validation",
        batch_size=32,
        seed=123,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224))

Found 1101 validated image filenames belonging to 35 classes.


In [ ]:
model.fit(train_gen, steps_per_epoch=20, 
                    epochs=75, validation_data=test_gen, 
                    validation_steps=20)

Epoch 1/75
20/20 [==============================] - 390s 19s/step - loss: 5.2672 - accuracy: 0.0479 - val_loss: 4.1902 - val_accuracy: 0.0766
Epoch 2/75
20/20 [==============================] - 232s 12s/step - loss: 4.1299 - accuracy: 0.0692 - val_loss: 4.4039 - val_accuracy: 0.0969
Epoch 3/75
20/20 [==============================] - 163s 8s/step - loss: 3.9867 - accuracy: 0.1017 - val_loss: 3.8810 - val_accuracy: 0.1125
Epoch 4/75
20/20 [==============================] - 127s 6s/step - loss: 3.6710 - accuracy: 0.1502 - val_loss: 3.7579 - val_accuracy: 0.1406
Epoch 5/75
20/20 [==============================] - 110s 6s/step - loss: 3.6106 - accuracy: 0.1526 - val_loss: 3.6663 - val_accuracy: 0.1000
Epoch 6/75
20/20 [==============================] - 92s 5s/step - loss: 3.4918 - accuracy: 0.1829 - val_loss: 3.8422 - val_accuracy: 0.1156
Epoch 7/75
20/20 [==============================] - 76s 4s/step - loss: 3.3993 - accuracy: 0.1729 - val_loss: 3.7677 - val_accuracy: 0.1000
Epoch 8/75
20

In [ ]:
#model.fit(train_gen, steps_per_epoch=15, 
                    #epochs=16, validation_data=test_gen, 
                    #validation_steps=15)